# Imports

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import gc
import random
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from pytorch_lightning import seed_everything, LightningModule, Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [ ]:
def seed_everything(seed=47):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False

seed_everything()
# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'Using device: {device}')

In [ ]:
train = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
train.head()

In [ ]:
META_FEATURES = [col for col in train if col not in ['Id','Pawpularity']]
train, y = train[META_FEATURES], train['Pawpularity']
test = test[META_FEATURES]

In [ ]:
class CustomDatasetGet(Dataset):
    def __init__(self, data: pd.core.frame.DataFrame, is_test: bool=False):
        self.is_test = is_test
        self.target = y.values
        self.features = data.values
    def __getitem__(self, idx):
        data = self.features[idx]
        if self.is_test:
            return torch.tensor(data, dtype=torch.float32)
        else:
            target = self.target[idx]
            return torch.tensor(data, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)
    def __len__(self):
        return len(self.features)

In [ ]:
def get_datasets(data: pd.core.frame.DataFrame, split: int=0.2):
    """
    Split the data into training and validation splits
    Make them into Torch Dataset format
    """
    # Shuffle the data
    data = data.sample(frac=1).reset_index(drop=True)
    # Split the data
    split_nb = int(split * len(data))
    train_split = data[split_nb:]
    val_split = data[:split_nb]
    
    # Make them Torch Datasets
    training_set = CustomDatasetGet(
        train_split,
        is_test=False
    )
    validation_set = CustomDatasetGet(
        val_split,
        is_test=False
    )
    return {'train': training_set, 'val' : validation_set}

In [ ]:
data_config = {
    'data': train,
    'split_pcent': 0.1,
    'data_ret': get_datasets,
    'num_workers': 1,
    'train_bs': 128,
    'val_bs': 128
}

# Lightning Model

In [ ]:
def fc_block(in_f, out_f):
    return nn.Sequential(
        nn.Linear(in_f, out_f),
        nn.ReLU(),
    )  

class TPSModel(LightningModule):
    def __init__(self,
                 input_size: int = data_config['data'].shape[1], 
                 classes: int = 1,
                 learning_rate: float = 1e-4,
                 data_config: dict = data_config
        ):
        super(TPSModel, self).__init__()
        
        if not data_config:
            raise ValueError("Data Config Cannot be empty")
        self.epoch_counter = 0
        self.data_config = data_config
        self.input_size = input_size
        self.learning_rate = learning_rate

        # Mode Architecture
        self.fc1 = fc_block(self.input_size, 128)
        self.fc2 = fc_block(128, 64)
        self.fc_relu = fc_block(192, 64)
        self.fc3 = fc_block(64, 64)
        self.out = nn.Linear(64, classes)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        # Model Compuatation Code
        #x = self.flatten(x)
        x1 = self.fc1(x)
        x = self.fc2(x1)
        x_cat = torch.cat((x, x1), dim=1)
        x = self.dropout(x_cat)
        x = self.fc_relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.out(x)
        return x

    def prepare_data(self):
        """
        Get the datasets related variables and functions from the data config dictionary
        Then use it to split data.
        """
        # Get stuff from dict
        data = self.data_config['data']
        split_pcent = self.data_config['split_pcent']
        data_ret_fn = self.data_config['data_ret']
        
        # Call the retriever function to split data and make datasets
        # Also extract the datasets from the returned dictionary
        dataset_cache = data_ret_fn(data, split_pcent)
        self.train_set = dataset_cache['train']
        self.val_set = dataset_cache['val']
        
    def train_dataloader(self):
        """
        Initializes and returns the training dataloader
        """
        num_workers = self.data_config['num_workers']
        train_bs = self.data_config['train_bs']
        train_loader = DataLoader(
            dataset = self.train_set,
            shuffle = True,
            batch_size = train_bs,
            num_workers = num_workers
        )
        return train_loader
        
    def val_dataloader(self):
        """
        Initializes and returns the validation dataloader
        """
        num_workers = self.data_config['num_workers']
        val_bs = self.data_config['val_bs']
        val_loader = DataLoader(
            dataset = self.val_set,
            shuffle = False,
            batch_size = val_bs,
            num_workers = num_workers,
        )
        return val_loader
    
    def training_step(self, batch, batch_idx):
        data, targets = batch
        data, targets = data.to(device), targets.to(device)
        outputs = self(data)
        loss = torch.sqrt(F.mse_loss(outputs.squeeze(1), targets))
        return {'loss': loss}

    def training_epoch_end(self, outputs):
        self.epoch_counter += 1
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        print(f'Epoch: {self.epoch_counter} \t Average train_loss: {avg_loss}')
    
    def validation_step(self, batch, batch_idx):
        data, targets = batch
        data, targets = data.to(device), targets.to(device)
        outputs = self(data)
        val_loss = torch.sqrt(F.mse_loss(outputs.squeeze(1), targets))
        self.log("val_loss", val_loss)
        return {'val_loss': val_loss}

    def validation_epoch_end(self, outputs):
        # 'outputs' is a list of dictionaries containing validation loss of each batch
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print(f'Epoch: {self.epoch_counter} \t Average val_loss: {avg_loss}')
        return {'val_loss': avg_loss}
        self.log(f"Average loss: {avg_loss}")
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

# Training

In [ ]:
model = TPSModel()
model = model.to(device)
es_scheduler = EarlyStopping(monitor="val_loss", patience=10, verbose=False, min_delta=0.0001, mode="min")
trainer = Trainer(max_epochs=100, gpus=1, callbacks=[es_scheduler])
trainer.fit(model)

# Submission

In [ ]:
test_dataset = CustomDatasetGet(
        test,
        is_test=True
    )
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
def prediction(model_):
    predictions = []
    model_.eval()
    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)
            inputs = inputs.to(torch.float32)
            model_ = model_.to(device)
            pred = model_(inputs)
            predictions.append(pred)
        return torch.tensor(predictions)

y_pred = prediction(model)

# submission
submission['Pawpularity'] = y_pred.cpu().detach().numpy()
submission.to_csv('submission.csv', index=None)
submission.head(3)